# 🧪 Test du Modèle d'Estimation Immobilière

Ce notebook contient les tests unitaires pour valider le modèle d'estimation des prix immobiliers au Maroc.

## Objectifs des tests :
1. Vérifier le chargement du modèle
2. Tester les prédictions sur plusieurs exemples d'appartements
3. Évaluer la précision par rapport aux prix réels estimés

## 1. Importation des bibliothèques

In [1]:
import unittest
import joblib
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## 2. Chargement du modèle

In [2]:
# Charger le modèle pré-entraîné
model = joblib.load('models/model.pkl')
print("✅ Modèle chargé avec succès")
print(f"Type du modèle : {type(model).__name__}")

✅ Modèle chargé avec succès
Type du modèle : RandomForestRegressor


## 3. Vérification des features attendues

In [3]:
# Afficher les features attendues par le modèle
print("Features attendues par le modèle :")
print(list(model.feature_names_in_))
print(f"\nNombre de features : {len(model.feature_names_in_)}")

Features attendues par le modèle :
['price_per_m2', 'surface_area', 'nb_baths', 'total_rooms', 'nb_rooms', 'Ascenseur', 'Parking', 'Climatisation', 'Terrasse', 'Chauffage', 'Concierge', 'Balcon']

Nombre de features : 12


## 4. Préparation des données de test

In [4]:
# Créer plusieurs exemples d'appartements pour tester le modèle
# Utiliser l'ordre exact des features utilisé lors de l'entraînement
test_data = pd.DataFrame({
    'price_per_m2': [15000, 12000, 18000, 10000, 20000],
    'surface_area': [80, 120, 60, 100, 50],
    'nb_baths': [2, 3, 1, 2, 1],
    'total_rooms': [4, 5, 3, 4, 2],
    'nb_rooms': [3, 4, 2, 3, 1],
    'Ascenseur': [1, 1, 0, 1, 1],
    'Parking': [1, 1, 0, 1, 0],
    'Climatisation': [1, 1, 1, 0, 1],
    'Terrasse': [0, 1, 0, 1, 0],
    'Chauffage': [1, 1, 0, 1, 1],
    'Concierge': [0, 1, 0, 0, 1],
    'Balcon': [1, 1, 1, 0, 1]
})

print("📋 Données de test créées :")
print(f"Nombre d'appartements à tester : {len(test_data)}")
print("\n" + "="*60)
print(test_data)

📋 Données de test créées :
Nombre d'appartements à tester : 5

   price_per_m2  surface_area  nb_baths  total_rooms  nb_rooms  Ascenseur  \
0         15000            80         2            4         3          1   
1         12000           120         3            5         4          1   
2         18000            60         1            3         2          0   
3         10000           100         2            4         3          1   
4         20000            50         1            2         1          1   

   Parking  Climatisation  Terrasse  Chauffage  Concierge  Balcon  
0        1              1         0          1          0       1  
1        1              1         1          1          1       1  
2        0              1         0          0          0       1  
3        1              0         1          1          0       0  
4        0              1         0          1          1       1  


## 5. Test de prédiction

In [5]:
# Effectuer les prédictions
predictions = model.predict(test_data)

print("🔮 Prédictions effectuées :")
print(f"Nombre de prédictions : {len(predictions)}")
print(f"Type des prédictions : {type(predictions[0])}")

# Afficher les prédictions brutes
print("\n📊 Prédictions brutes :")
for i, pred in enumerate(predictions):
    print(f"Appartement {i+1}: {pred:,.2f} MAD")

🔮 Prédictions effectuées :
Nombre de prédictions : 5
Type des prédictions : <class 'numpy.float64'>

📊 Prédictions brutes :
Appartement 1: 1,184,840.00 MAD
Appartement 2: 1,420,950.00 MAD
Appartement 3: 1,095,632.50 MAD
Appartement 4: 1,001,176.00 MAD
Appartement 5: 973,360.00 MAD


## 6. Comparaison avec les prix réels estimés

In [6]:
# Prix estimés réels pour chaque appartement (exemples basés sur le marché marocain)
real_prices = [1200000, 1440000, 1080000, 1000000, 1000000]  # MAD

print("🏠 ANALYSE COMPARATIVE DES PRÉDICTIONS")
print("="*70)

# Analyser chaque appartement
for i in range(len(test_data)):
    pred_price = predictions[i]
    real_price = real_prices[i]
    difference = abs(pred_price - real_price)
    percentage_diff = (difference / real_price) * 100
    
    print(f"\n📋 APPARTEMENT {i+1}:")
    print(f"   📐 Surface: {test_data.iloc[i]['surface_area']}m² | 🛏️ Chambres: {test_data.iloc[i]['nb_rooms']} | 🚿 SDB: {test_data.iloc[i]['nb_baths']}")
    
    # Afficher les équipements
    equipments = []
    if test_data.iloc[i]['Ascenseur']: equipments.append('Ascenseur')
    if test_data.iloc[i]['Parking']: equipments.append('Parking')
    if test_data.iloc[i]['Climatisation']: equipments.append('Climatisation')
    if test_data.iloc[i]['Terrasse']: equipments.append('Terrasse')
    if test_data.iloc[i]['Chauffage']: equipments.append('Chauffage')
    if test_data.iloc[i]['Concierge']: equipments.append('Concierge')
    if test_data.iloc[i]['Balcon']: equipments.append('Balcon')
    
    print(f"   🏢 Équipements: {', '.join(equipments) if equipments else 'Aucun'}")
    print(f"   🤖 Prix prédit     : {pred_price:,.2f} MAD")
    print(f"   🏷️  Prix réel estimé: {real_price:,.2f} MAD")
    print(f"   📊 Différence      : {difference:,.2f} MAD ({percentage_diff:.1f}%)")
    
    # Évaluation de la précision
    if percentage_diff <= 5:
        print("   🎯 Prédiction EXCELLENTE (≤5% d'écart)")
    elif percentage_diff <= 10:
        print("   ✅ Prédiction TRÈS BONNE (≤10% d'écart)")
    elif percentage_diff <= 20:
        print("   🟡 Prédiction ACCEPTABLE (≤20% d'écart)")
    else:
        print("   ❌ Prédiction À AMÉLIORER (>20% d'écart)")

🏠 ANALYSE COMPARATIVE DES PRÉDICTIONS

📋 APPARTEMENT 1:
   📐 Surface: 80m² | 🛏️ Chambres: 3 | 🚿 SDB: 2
   🏢 Équipements: Ascenseur, Parking, Climatisation, Chauffage, Balcon
   🤖 Prix prédit     : 1,184,840.00 MAD
   🏷️  Prix réel estimé: 1,200,000.00 MAD
   📊 Différence      : 15,160.00 MAD (1.3%)
   🎯 Prédiction EXCELLENTE (≤5% d'écart)

📋 APPARTEMENT 2:
   📐 Surface: 120m² | 🛏️ Chambres: 4 | 🚿 SDB: 3
   🏢 Équipements: Ascenseur, Parking, Climatisation, Terrasse, Chauffage, Concierge, Balcon
   🤖 Prix prédit     : 1,420,950.00 MAD
   🏷️  Prix réel estimé: 1,440,000.00 MAD
   📊 Différence      : 19,050.00 MAD (1.3%)
   🎯 Prédiction EXCELLENTE (≤5% d'écart)

📋 APPARTEMENT 3:
   📐 Surface: 60m² | 🛏️ Chambres: 2 | 🚿 SDB: 1
   🏢 Équipements: Climatisation, Balcon
   🤖 Prix prédit     : 1,095,632.50 MAD
   🏷️  Prix réel estimé: 1,080,000.00 MAD
   📊 Différence      : 15,632.50 MAD (1.4%)
   🎯 Prédiction EXCELLENTE (≤5% d'écart)

📋 APPARTEMENT 4:
   📐 Surface: 100m² | 🛏️ Chambres: 3 | 🚿 SDB

## 7. Statistiques globales

In [7]:
# Calculer les statistiques globales
differences = [abs(predictions[i] - real_prices[i]) for i in range(len(test_data))]
percentage_diffs = [abs(predictions[i] - real_prices[i]) / real_prices[i] * 100 for i in range(len(test_data))]

avg_diff = sum(differences) / len(differences)
avg_percentage = sum(percentage_diffs) / len(percentage_diffs)
max_diff = max(differences)
min_diff = min(differences)
max_percentage = max(percentage_diffs)
min_percentage = min(percentage_diffs)

print("📊 STATISTIQUES GLOBALES DE PERFORMANCE")
print("="*50)
print(f"📈 Écart moyen        : {avg_diff:,.2f} MAD ({avg_percentage:.1f}%)")
print(f"📉 Écart minimum      : {min_diff:,.2f} MAD ({min_percentage:.1f}%)")
print(f"📊 Écart maximum      : {max_diff:,.2f} MAD ({max_percentage:.1f}%)")
print(f"🎯 Précision moyenne  : {100 - avg_percentage:.1f}%")

# Évaluation globale
if avg_percentage <= 5:
    print("\n🏆 ÉVALUATION GLOBALE : MODÈLE EXCELLENT")
elif avg_percentage <= 10:
    print("\n✅ ÉVALUATION GLOBALE : MODÈLE TRÈS BON")
elif avg_percentage <= 20:
    print("\n🟡 ÉVALUATION GLOBALE : MODÈLE ACCEPTABLE")
else:
    print("\n❌ ÉVALUATION GLOBALE : MODÈLE À AMÉLIORER")

print("\n" + "="*50)
print("✅ TESTS TERMINÉS AVEC SUCCÈS")

📊 STATISTIQUES GLOBALES DE PERFORMANCE
📈 Écart moyen        : 15,531.70 MAD (1.4%)
📉 Écart minimum      : 1,176.00 MAD (0.1%)
📊 Écart maximum      : 26,640.00 MAD (2.7%)
🎯 Précision moyenne  : 98.6%

🏆 ÉVALUATION GLOBALE : MODÈLE EXCELLENT

✅ TESTS TERMINÉS AVEC SUCCÈS


## 8. Test unitaire programmatique

In [8]:
# Exécuter les tests unitaires
class TestModel(unittest.TestCase):
    def test_model_load(self):
        """Test si le modèle peut être chargé"""
        self.assertIsNotNone(model)
        print("✅ Test chargement modèle : RÉUSSI")
    
    def test_prediction_shape(self):
        """Test si les prédictions ont la bonne forme"""
        self.assertEqual(predictions.shape, (len(test_data),))
        print("✅ Test forme prédictions : RÉUSSI")
    
    def test_prediction_type(self):
        """Test si les prédictions sont du bon type"""
        self.assertTrue(all(isinstance(p, (float, int)) for p in predictions))
        print("✅ Test type prédictions : RÉUSSI")
    
    def test_prediction_range(self):
        """Test si les prédictions sont dans une plage raisonnable"""
        for pred in predictions:
            self.assertGreater(pred, 0, "Prix doit être positif")
            self.assertLess(pred, 10000000, "Prix doit être réaliste")
        print("✅ Test plage prédictions : RÉUSSI")

# Exécuter les tests
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestModel)
    runner = unittest.TextTestRunner(verbosity=0)
    result = runner.run(suite)
    
    if result.wasSuccessful():
        print("\n🎉 TOUS LES TESTS UNITAIRES ONT RÉUSSI !")
    else:
        print("\n❌ CERTAINS TESTS ONT ÉCHOUÉ")

----------------------------------------------------------------------
Ran 4 tests in 0.000s

OK


✅ Test chargement modèle : RÉUSSI
✅ Test plage prédictions : RÉUSSI
✅ Test forme prédictions : RÉUSSI
✅ Test type prédictions : RÉUSSI

🎉 TOUS LES TESTS UNITAIRES ONT RÉUSSI !
